In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import os
import matplotlib.pyplot as plt
import pickle
from Topo import Topo

from sklearn.preprocessing import StandardScaler

In [2]:
%load_ext autoreload

%autoreload 2

In [25]:
netlist_name = 'jpeg_300_off_low_GENUS'

with open('d_dict.pkl', 'rb') as f:
    new_d_dict = pickle.load(f)

with open('delay_dict.pkl', 'rb') as f:
    delay_dict = pickle.load(f)

G = nx.read_gpickle(f"{netlist_name}.gpickle")

G.remove_edges_from(list(nx.selfloop_edges(G)))

col_row = np.load(f"/home/zluo/output/{netlist_name}.npy")

col = col_row[0]
row = col_row[1]
edgs = np.array([col, row]).T

with open("net_dict/jpeg_300_off_low_GENUSnet.pkl", "rb") as f:
    net_dict = pickle.load(f)

In [61]:
netlist_name = 'jpeg_300_off_low_GENUS_o'

with open('delay_dict_jpeg_300_off_low_GENUS_o.pkl', 'rb') as f:
    delay_dict = pickle.load(f)
    
with open('jpeg_300_off_low_GENUS_oinst.pkl', 'rb') as f:
    dinst = pickle.load(f)

G = nx.read_gpickle(f"{netlist_name}.gpickle")

col_row = np.load(f"{netlist_name}.npy")

col = col_row[0]
row = col_row[1]
edgs = np.array([col, row]).T

with open(f"{netlist_name}net.pkl", "rb") as f:
    net_dict = pickle.load(f)

In [26]:
start_pt_dict = {net_dict[net][0][0]:net for net in net_dict.keys()}

with open("jpeg_300_off_low_GENUS_WL.rpt", "r") as f:
    ground_truth = [val[:-1].split(", ") for val in f.readlines()][1:]
    
wirelength = dict(ground_truth)

w_dict = dict()

for key in start_pt_dict.keys():
    net = start_pt_dict[key]
    
    try:
        w_dict[key] = float(wirelength[net])
    
    except:
        continue
        
G = G.subgraph(list(w_dict.keys()))

In [27]:
#instances as nodes
X = []
for node in G:
    x = []
    x.append(G.in_degree()[node])
    x.append(G.out_degree()[node])
    x.append(delay_dict[node])
    X.append(x)

idx_dict = dict()
nodelist = list(G.nodes)
for idx in range(len(nodelist)):
    idx_dict[nodelist[idx]] = idx
    
edgs = [[idx_dict[tp[0]], idx_dict[tp[1]]] for tp in list(G.edges)]

encoder = StandardScaler()
X = encoder.fit_transform(X)

np.save("to_gnn/train_inst_X.npy", X)
np.save("to_gnn/edge_index_train_inst.npy", edgs)

In [28]:
#build hyperedge bodes
G_hyper = nx.DiGraph(G)
to_add_start = 0
X_hyper = []
Y = []
edgs_inst_to_net = []
edgs_net_to_inst = []
for node in nodelist:
    w_l = w_dict[node]
    o_d = G.out_degree()[node]
    X_hyper.append([o_d])
    Y.append(w_l)
    
    edgs_inst_to_net.append([idx_dict[node], to_add_start])
    for sub_node in G.neighbors(node):
        edgs_net_to_inst.append([to_add_start, idx_dict[sub_node]])
        
    to_add_start += 1

In [58]:
edgs_inst_to_net

[[0, 0],
 [1, 1],
 [2, 2],
 [3, 3],
 [4, 4],
 [5, 5],
 [6, 6],
 [7, 7],
 [8, 8],
 [9, 9],
 [10, 10],
 [11, 11],
 [12, 12],
 [13, 13],
 [14, 14],
 [15, 15],
 [16, 16],
 [17, 17],
 [18, 18],
 [19, 19],
 [20, 20],
 [21, 21],
 [22, 22],
 [23, 23],
 [24, 24],
 [25, 25],
 [26, 26],
 [27, 27],
 [28, 28],
 [29, 29],
 [30, 30],
 [31, 31],
 [32, 32],
 [33, 33],
 [34, 34],
 [35, 35],
 [36, 36],
 [37, 37],
 [38, 38],
 [39, 39],
 [40, 40],
 [41, 41],
 [42, 42],
 [43, 43],
 [44, 44],
 [45, 45],
 [46, 46],
 [47, 47],
 [48, 48],
 [49, 49],
 [50, 50],
 [51, 51],
 [52, 52],
 [53, 53],
 [54, 54],
 [55, 55],
 [56, 56],
 [57, 57],
 [58, 58],
 [59, 59],
 [60, 60],
 [61, 61],
 [62, 62],
 [63, 63],
 [64, 64],
 [65, 65],
 [66, 66],
 [67, 67],
 [68, 68],
 [69, 69],
 [70, 70],
 [71, 71],
 [72, 72],
 [73, 73],
 [74, 74],
 [75, 75],
 [76, 76],
 [77, 77],
 [78, 78],
 [79, 79],
 [80, 80],
 [81, 81],
 [82, 82],
 [83, 83],
 [84, 84],
 [85, 85],
 [86, 86],
 [87, 87],
 [88, 88],
 [89, 89],
 [90, 90],
 [91, 91],
 [92, 92

In [29]:
encoder = StandardScaler()
X_hyper = encoder.fit_transform(X_hyper)

np.save("to_gnn/train_net_X.npy", X_hyper)
np.save("to_gnn/edgs_inst_to_net.npy", edgs_inst_to_net)
np.save("to_gnn/edgs_net_to_inst.npy", edgs_net_to_inst)
np.save("to_gnn/net_Y.npy", Y)

In [50]:
X[:10]

array([[-0.59451473, -0.03083091, -0.91616952],
       [ 1.46259341, -0.08417065, -0.94346949],
       [-0.59451473, -0.03083091, -0.91616952],
       [ 1.46259341, -0.08417065, -0.94346949],
       [-0.59451473, -0.03083091, -0.91616952],
       [ 1.46259341, -0.08417065, -0.94346949],
       [-0.59451473, -0.03083091, -0.91616952],
       [ 1.46259341, -0.08417065, -0.94346949],
       [-0.59451473, -0.03083091, -0.91616952],
       [ 1.46259341, -0.08417065, -0.94346949]])

In [54]:
idx_dict

{74780: 0,
 74918: 1,
 74781: 2,
 74911: 3,
 74782: 4,
 74921: 5,
 74783: 6,
 74914: 7,
 74784: 8,
 74909: 9,
 74785: 10,
 74923: 11,
 74786: 12,
 74925: 13,
 74787: 14,
 74907: 15,
 74788: 16,
 74915: 17,
 74789: 18,
 74908: 19,
 74790: 20,
 74910: 21,
 13: 22,
 74950: 23,
 74949: 24,
 74948: 25,
 74947: 26,
 74946: 27,
 74941: 28,
 74940: 29,
 74939: 30,
 74938: 31,
 74792: 32,
 74791: 33,
 74768: 34,
 74767: 35,
 74766: 36,
 74765: 37,
 74764: 38,
 74759: 39,
 74758: 40,
 74757: 41,
 74756: 42,
 74603: 43,
 74602: 44,
 74601: 45,
 74600: 46,
 74599: 47,
 74598: 48,
 74597: 49,
 74596: 50,
 74595: 51,
 74594: 52,
 74593: 53,
 74592: 54,
 74591: 55,
 74590: 56,
 74589: 57,
 74577: 58,
 74576: 59,
 74575: 60,
 74574: 61,
 74573: 62,
 74568: 63,
 74567: 64,
 74566: 65,
 74565: 66,
 74413: 67,
 74412: 68,
 74411: 69,
 74410: 70,
 74409: 71,
 74408: 72,
 74407: 73,
 74406: 74,
 74405: 75,
 74404: 76,
 74403: 77,
 74402: 78,
 74401: 79,
 74400: 80,
 74399: 81,
 74387: 82,
 74386: 83,
 7438

In [42]:
G.nodes[74780]

{'inst_name': 'rle_rz4_ampo_reg[0]', 'type': 'inst'}

In [55]:
w_dict[74918]

0.504

In [45]:
idx_dict[74781]

2

In [35]:
net_dict

{'amp[0]': [[74780, 2], [74780, 74918]],
 'amp[10]': [[74781, 3], [74781, 74911]],
 'amp[1]': [[74782, 4], [74782, 74921]],
 'amp[2]': [[74783, 5], [74783, 74914]],
 'amp[3]': [[74784, 6], [74784, 74909]],
 'amp[4]': [[74785, 7], [74785, 74923]],
 'amp[5]': [[74786, 8], [74786, 74925]],
 'amp[6]': [[74787, 9], [74787, 74907]],
 'amp[7]': [[74788, 10], [74788, 74915]],
 'amp[8]': [[74789, 11], [74789, 74908]],
 'amp[9]': [[74790, 12], [74790, 74910]],
 'clk': [[13, 74950],
  [13, 74949],
  [13, 74948],
  [13, 74947],
  [13, 74946],
  [13, 74945],
  [13, 74944],
  [13, 74943],
  [13, 74942],
  [13, 74941],
  [13, 74940],
  [13, 74939],
  [13, 74938],
  [13, 74937],
  [13, 74936],
  [13, 74935],
  [13, 74934],
  [13, 74792],
  [13, 74791],
  [13, 74790],
  [13, 74789],
  [13, 74788],
  [13, 74787],
  [13, 74786],
  [13, 74785],
  [13, 74784],
  [13, 74783],
  [13, 74782],
  [13, 74781],
  [13, 74780],
  [13, 74779],
  [13, 74778],
  [13, 74777],
  [13, 74776],
  [13, 74775],
  [13, 74774]